In [1]:
import pandas as pd
import numpy as np
import spacy
from spacy.lang.fr import French
from spacy.tokenizer import Tokenizer
from spacy.tokens import Doc
from spacy.vocab import Vocab
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from textblob import TextBlob
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

In [640]:
product_data = pd.read_csv ('/Users/vincentsalamand/Downloads/products_leclerc.csv')
print(len(product_data))
foods = pd.read_csv('/Users/vincentsalamand/Downloads/food_export.csv')
print(len(foods))
food_categories = pd.read_csv('/Users/vincentsalamand/Downloads/category_export.csv')
print(len(food_categories))
product_categories = product_data.shelter_parent.unique()
print(len(product_categories))
mapping_categories = pd.read_csv('/Users/vincentsalamand/Downloads/category_mapping.csv')
mapping_categories.shelter = [name[:-2] for name in mapping_categories.shelter]

8173
1096
47
44


8173

In [337]:
measurements = {"g": ["gr.", "gr", "g", "g.", "gramme"], "kg": ["kg.", "kg", "kilogramme"], "mg": ["mg.", "mg", "miligramme"],
                "ml": ["ml.", "ml", "mililitre"], "cl": ["cl.", "cl", "centilitre"], "l": ["l.", "l", "litre"],
                "cuillère à soupe": ["c. à soupe", "cuillère à soupe", "cs", "cas", "càs", "c. à table", "cuillère à table"],
                "cuillère à café": ["c. à café", "cuillère à café", "cc", "cac", "càc", "c. à thé", "cuillère à thé"],
                "tasse": ["tasse"], "bol": ["bol"], "verre": ["verre"], "filet": ["filet"], "zeste": ["zeste"], "pièce": ["pièce"],
                "noisette": ["noisette"], "noix": ["noix"], "pincée": ["pincée"], "pointe":["pointe"], "poignée": ["poignée"],
                "feuille": ["feuille"], "branche": ["branche"], "gousse": ["gousse"], "tranche": ["tranche"], "cube": ["cube"],
                "boîte": ["boîte"], "barquette": ["barquette"], "pot": ["pot"], "bâtonnet": ["bâtonnet"], "boule": ["boule"],
                "rouleau": ["rouleau"], "p": ["p"]}

units = [item for sublist in measurements.values() for item in sublist]


In [338]:
quantities = { "½": 0.5, "1/2": 0.5, "1½": 1.5, "1 1/2": 0.5, "⅓": 0.33, "1/3": 0.33, "¼": 0.25, "1/5": 0.2, "¼": 0.25, "un demi": 0.5, "une demi": 0.5,"un et demi": 1.5,
              "tiers": 0.33, "quart": 0.25, "zero": 0, "deux": 2, "trois": 3, "quatre": 4,"cinq": 5, "six": 6, "sept": 7,
              "huit": 8, "neuf": 9, "dix": 10, "onze": 11, "douze": 12, "treize": 13, "quatorze": 14, "quinze": 15, "seize": 16,
              "dix-sept": 17, "dix-huit": 18, "dix-neuf": 19, "vingt": 20, "trente": 30, "quarante": 40, "cinquante": 50,
              "soixante": 60, "soixante-dix": 70, "quatre-vingt": 80, "quatre-vingt-dix": 90, "dizaine": 10, "une dizaine": 10,
              "douzaine": 12, "une douzaine": 12, "demi-douzaine": 6, "une demi-douzaine": 6, "vingtaine": 20, "une vingtaine": 20,
              "trentaine": 30, "quarantaine": 40, "cinquantaine": 50, "centaine": 100, "une centaine": 100, "cent": 100,
              "un": 1, "une": 1}

In [339]:
char_list = [i for i in "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZéèàâêïù▪️"]


In [340]:
data = pd.DataFrame()
data['quantity_match'] = ""
data['unit_match'] = ""
data['food_match'] = ""
data['price_match'] = ""
data['price_per_unit_match'] = ""
data = pd.concat([data, product_data], axis=1)
data = data.drop(columns=['image_url', 'origin', 'product_id', 'url'])
pd.set_option('display.max_colwidth', 100)

data.head()

,quantity_match,unit_match,food_match,price_match,price_per_unit_match,brand,description,price,price_per_unit,promo_price,shelter_child,shelter_main,shelter_parent
0,NaN,NaN,NaN,NaN,NaN,NaN,Plateau grill Tendre et Plus Extra tendre - Herbes x4 -650g,8.84 €,13.60 € / kg,8.84 €,Saucisses et Grillades,Viandes Poissons,Boucherie
1,NaN,NaN,NaN,NaN,NaN,NaN,Brochette porc Tendre Et Plus Extra tendre x4 - 320g,3.89 €,12.16 € / kg,3.89 €,Saucisses et Grillades,Viandes Poissons,Boucherie
2,NaN,NaN,NaN,NaN,NaN,NaN,Saucisses aux herbes Ferial Avec Chipolata,2.97 €,9.00 € / kg,2.97 €,Saucisses et Grillades,Viandes Poissons,Boucherie
3,NaN,NaN,NaN,NaN,NaN,NaN,Saucisse Chipolatas Bio x5 - VPF - 275g,4.07 €,14.80 € / kg,4.07 €,Saucisses et Grillades,Viandes Poissons,Boucherie
4,NaN,NaN,NaN,NaN,NaN,Bigard,Chipolatas label rouge Bigard x6 - 330g,3.55 €,10.76 € / kg,3.55 €,Saucisses et Grillades,Viandes Poissons,Boucherie


In [341]:
data['price_match'] = data.price.apply(lambda x: float(x.split()[0]))
data['price_per_unit_match'] = data.price_per_unit.apply(lambda x: float(x.split()[0]))
data.quantity_match = round(data.price_match / data.price_per_unit_match, 3)

In [343]:
nlp = French()
#products = list(nlp.pipe(data.description))
#price_per_unit = list(nlp.pipe(data.price_per_unit))

In [344]:
def convert_unit(value):
    results = []
    result = [unit for unit, item in measurements.items() if value in item]
    if len(result) > 0:
        return result[0]

In [345]:
measurement_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
measurement_patterns = list(nlp.tokenizer.pipe(list(units)))
measurement_matcher.add("MEASUREMENT_PATTERN", None, *measurement_patterns)
tokenizer = Tokenizer(nlp.vocab)

measurement_parser = []
price_per_unit = list(nlp.pipe(data.price_per_unit))

for line in price_per_unit:
    line = tokenizer(re.sub('[0-9]', ' ', line.text))
    matches = measurement_matcher(line)
    elements = []
    if len(matches) > 0:
        for match_id, start, end in matches:
            span = line[start:end]
            elements.append(span.text.lower())
        measurement_parser.append(convert_unit(elements[0]))
    else:
        text_blob_object = TextBlob(line.text)
        singular_line = ' '.join(text_blob_object.words.singularize())
        matches = measurement_matcher(tokenizer(singular_line))
        if len(matches) > 0:
            for match_id, start, end in matches:
                span = tokenizer(singular_line)[start:end]
                elements.append(span.text.lower())
            measurement_parser.append(convert_unit(elements[0]))
        else:
            plural_line = ' '.join(text_blob_object.words.pluralize())
            matches = measurement_matcher(tokenizer(plural_line))
            if len(matches) > 0:
                for match_id, start, end in matches:
                    span = tokenizer(plural_line)[start:end]
                    elements.append(span.text.lower())
                measurement_parser.append(convert_unit(elements[0]))
            else:
                 measurement_parser.append("")

data['unit_match'] = measurement_parser

In [657]:
#matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
#patterns = list(nlp.tokenizer.pipe(foods.name))
#matcher.add("FOOD_PATTERN", None, *patterns)
tokenizer = Tokenizer(nlp.vocab)

for product_category in product_categories:
    filter_data = data[data.shelter_parent == product_category].description
    products = list(nlp.pipe(filter_data))
    
    mapping = mapping_categories[mapping_categories.shelter == product_category]["category ids"].str.split(',', expand=True)
    if not mapping.isnull().values.any():
        cat_ids = [int(ids[1]) for ids in mapping.iteritems()]
        related_foods = foods[foods.category_id.isin(cat_ids)]
    else:
        related_foods = foods
    
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
    patterns = list(nlp.tokenizer.pipe(related_foods.name))
    matcher.add("FOOD_PATTERN", None, *patterns)
    
    food_parser = []
    
    for line in products:
        matches = matcher(line)
        elements = []
        if len(matches) > 0:
            for match_id, start, end in matches:
                span = line[start:end]
                elements.append(span.text)
            food_parser.append(max(elements, key=len).lower())
        else:
            text_blob_object = TextBlob(line.text)
            singular_line = ' '.join(text_blob_object.words.singularize())
            matches = matcher(tokenizer(singular_line))
            if len(matches) > 0:
                for match_id, start, end in matches:
                    span = tokenizer(singular_line)[start:end]
                    elements.append(span.text)
                food_parser.append(max(elements, key=len).lower())
            else:
                plural_line = ' '.join(text_blob_object.words.pluralize())
                matches = matcher(tokenizer(plural_line))
                if len(matches) > 0:
                    for match_id, start, end in matches:
                        span = tokenizer(plural_line)[start:end]
                        elements.append(span.text)
                    food_parser.append(max(elements, key=len).lower())
                else:
                    ratios = process.extractOne(line.text.lower(), related_foods.name)
                    if len(ratios) > 0 and ratios[1] > 85:
                        food_parser.append(ratios[0])
                    else:
                        food_parser.append("")

    data.loc[data.shelter_parent == product_category, 'food_match'] = food_parser

len(data[data.food_match == '']) / len(data)

In [672]:
print(len(data[data.food_match == '']) / len(data)*100)
data

11.476813899424936


,quantity_match,unit_match,food_match,price_match,price_per_unit_match,brand,description,price,price_per_unit,promo_price,shelter_child,shelter_main,shelter_parent
0,0.650,kg,saucisse aux herbes,8.84,13.60,NaN,Plateau grill Tendre et Plus Extra tendre - Herbes x4 -650g,8.84 €,13.60 € / kg,8.84 €,Saucisses et Grillades,Viandes Poissons,Boucherie
1,0.320,kg,brochette,3.89,12.16,NaN,Brochette porc Tendre Et Plus Extra tendre x4 - 320g,3.89 €,12.16 € / kg,3.89 €,Saucisses et Grillades,Viandes Poissons,Boucherie
2,0.330,kg,chipolata,2.97,9.00,NaN,Saucisses aux herbes Ferial Avec Chipolata,2.97 €,9.00 € / kg,2.97 €,Saucisses et Grillades,Viandes Poissons,Boucherie
3,0.275,kg,saucisse,4.07,14.80,NaN,Saucisse Chipolatas Bio x5 - VPF - 275g,4.07 €,14.80 € / kg,4.07 €,Saucisses et Grillades,Viandes Poissons,Boucherie
4,0.330,kg,chipolata,3.55,10.76,Bigard,Chipolatas label rouge Bigard x6 - 330g,3.55 €,10.76 € / kg,3.55 €,Saucisses et Grillades,Viandes Poissons,Boucherie
5,0.330,kg,chipolata,2.99,9.06,Férial,Chipolatas Férial 6x - 55g,2.99 €,9.06 € / kg,2.99 €,Saucisses et Grillades,Viandes Poissons,Boucherie
6,0.330,kg,chipolata,3.10,9.39,Socopa,Chipolatas superieure Socopa Sous atmosphère x6 - 330g,3.10 €,9.39 € / kg,3.10 €,Saucisses et Grillades,Viandes Poissons,Boucherie
7,0.330,kg,chipolata,2.80,8.48,NaN,Chipolatas Tendre Et Plus x6 - 330g,2.80 €,8.48 € / kg,2.80 €,Saucisses et Grillades,Viandes Poissons,Boucherie
8,0.330,kg,chipolata,3.69,11.18,NaN,Chipolatas supérieures x6 - 330g,3.69 €,11.18 € / kg,3.69 €,Saucisses et Grillades,Viandes Poissons,Boucherie
9,1.200,kg,chipolata,8.03,6.69,NaN,Chipolatas boyau naturel - VPF x24 -,8.03 €,6.69 € / kg,8.03 €,Saucisses et Grillades,Viandes Poissons,Boucherie


In [667]:
data[data.food_match == '']

,quantity_match,unit_match,food_match,price_match,price_per_unit_match,brand,description,price,price_per_unit,promo_price,shelter_child,shelter_main,shelter_parent
17,0.320,kg,,2.99,9.34,NaN,Mini à griller Tendre et Plus Tapas nature x8 - 320g,2.99 €,9.34 € / kg,2.99 €,Saucisses et Grillades,Viandes Poissons,Boucherie
145,0.400,kg,,2.71,6.78,NaN,Farce à legumes Ferial 400g,2.71 €,6.78 € / kg,2.71 €,Viande hachée et Farce,Viandes Poissons,Boucherie
148,0.120,kg,,1.41,11.75,Férial,Sauce Forestière Férial 120g,1.41 €,11.75 € / kg,1.41 €,Sauces,Viandes Poissons,Boucherie
150,0.320,kg,,3.98,12.44,NaN,Saucisses Maître Coq Volaille nature x4 - 320g,3.98 €,12.44 € / kg,3.98 €,Grillades et Saucisses,Viandes Poissons,Volailles et Gibiers
151,0.320,kg,,3.98,12.44,NaN,Saucisses Maître Coq Volaille aux herbes x4 - 320g,3.98 €,12.44 € / kg,3.98 €,Grillades et Saucisses,Viandes Poissons,Volailles et Gibiers
255,0.200,kg,,1.45,7.25,NaN,Cordon bleu Côté Croc Comté - 200g,1.45 €,7.25 € / kg,1.45 €,"Panés, Nuggets et Grignottes",Viandes Poissons,Volailles et Gibiers
263,0.200,kg,,2.14,10.70,Le Gaulois,Croq pizza Le Gaulois x2 - 200g,2.14 €,10.70 € / kg,2.14 €,"Panés, Nuggets et Grignottes",Viandes Poissons,Volailles et Gibiers
264,0.200,kg,,2.02,10.10,Le Gaulois,Croq Extra Fromage Le Gaulois x2 - 200g,2.02 €,10.10 € / kg,2.02 €,"Panés, Nuggets et Grignottes",Viandes Poissons,Volailles et Gibiers
266,0.200,kg,,2.20,11.00,NaN,Rosti Le gaulois 200g - x2,2.20 €,11.00 € / kg,2.20 €,"Panés, Nuggets et Grignottes",Viandes Poissons,Volailles et Gibiers
269,0.200,kg,,1.43,7.15,NaN,Crousty Croc' Fromage Père Dodu x2 - 200g,1.43 €,7.15 € / kg,1.43 €,"Panés, Nuggets et Grignottes",Viandes Poissons,Volailles et Gibiers


In [673]:
len(foods)

1096

In [693]:
len(data.food_match.unique())

739

In [707]:
print(len(set(foods.name.tolist()) - set(data.food_match.tolist())))
set(foods.name.tolist()) - set(data.food_match.tolist())

385


{"Bleu d'Auvergne",
 'Haricot beurre',
 'Ossau iraty',
 'Pouligny saint pierre',
 'acras de morue',
 'agar agar',
 'aiguillette de poulet',
 'aileron de poulet',
 'airelle',
 'algue',
 'amande effilée',
 'amande en poudre',
 'anchoïade',
 'anguille',
 'artichaut mariné',
 'arôme',
 'asperge blanche',
 'açaï',
 'aïoli',
 'bagel',
 'baguette',
 'baguette céréales',
 'baguette tradition',
 'baguette viennoise',
 'baie de Goji',
 'bar',
 'barre marbrée',
 'blanc de poulet',
 'blanc de volaille ',
 'blanquette de dinde',
 'blette',
 'blini',
 'blé précuit',
 'boisson au riz',
 'boisson noisette',
 "boisson à l'avoine",
 "boisson à l'épeautre",
 'boite de thon',
 'bouillon de poisson',
 'boulette de viande',
 'bresaola',
 'brillat Savarin',
 'brique de brebis ',
 'brochet',
 'brochette de boeuf',
 'brochette de porc',
 'brochette de poulet',
 'brochette de veau',
 'brugnon',
 'burrata',
 'cabecou',
 'cajun',
 'cake anglais',
 'cantal',
 'carambole',
 'cardamome',
 'cardon',
 'carrelet',
 'ca